# Download data
## Install Kaggle

In [1]:
!pip install kaggle

     |████████████████████████████████| 59 kB 5.4 MB/s eta 0:00:011
     |████████████████████████████████| 118 kB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 8.9 MB/s  eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73274 sha256=6415a396e7b60a7bd14977350de6228cef55068fd5df63227ee670c81bc5a08a
  Stored in directory: /home/ec2-user/.cache/pip/wheels/cf/aa/f0/ed1179bbcd729b29d0dfda59826fb3b55f0a4a0c3f713c1c82
  Created wheel for python-slugify: filename=python_slugify-4.0.1-py2.py3-none-any.whl size=6767 sha256=68909f4a83dbda6174a92e8a1c3212f2fc00bda856ef6476556e150b3677be6d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/72/e6/db/122611605e60148f54ee2abaca98b2bbeafc6e22486a867bad
  Created wheel for slugify: filename=slugify-0.0.1-py3-none-any.whl size=1908 sha256=4be3dc4e013505590ead3ad2854a25b718563e9d887c234e3240539b0c1f202a
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7d/51/b0/c584cbdd0a8fc685d68677e58cde

### Kaggle Settings

Before running the kaggle download, kaggle configurations should saved locally in the notebook using a terminal prompt. 
```
sh-4.2$ history
sh-4.2$ mkdir ~/.kaggle
sh-4.2$ vi kaggle.json //add the kaggle credentials
sh-4.2$ mv kaggle.json ~/.kaggle/kaggle.json
sh-4.2$ chmod 600 ~/.kaggle/kaggle.json
sh-4.2$ kaggle
```

In [5]:
!kaggle datasets download --force  janiobachmann/bank-marketing-dataset

  0%|                                                | 0.00/142k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 142k/142k [00:00<00:00, 20.9MB/s]


In [6]:
!unzip bank-marketing-dataset.zip

Archive:  bank-marketing-dataset.zip
  inflating: bank.csv                


# Prepare Customer Data


## Imports


In [2]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time

In [2]:
df=pd.read_csv("bank.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


### Add a unique customer ID to the data

In [72]:
df['USER_ID'] = np.arange(len(df))+30908
# cols = df.columns.tolist()
# cols = cols[-1:] + cols[:-1]

## max 5 user attributes, get rid of some columns not very useful 
df_custmer = df[['USER_ID','deposit','age', 'job','marital','education']]
df_custmer.to_csv('customer11k.csv')
df_custmer.head()

,USER_ID,deposit,age,job,marital,education
0,30908,yes,59,admin.,married,secondary
1,30909,yes,56,admin.,married,secondary
2,30910,yes,41,technician,married,secondary
3,30911,yes,55,services,married,secondary
4,30912,yes,54,admin.,married,tertiary


## Upload customer data to S3

In [2]:
bucket ='personalize-custdata'      # replace with the name of your S3 bucket
filename ='customer11k.csv'
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

## Create personalize runtime

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### TODO: DELTE THIS BLOCK LATER

In [4]:
#personalize.list_schemas()
interactions_schema_arn = 'arn:aws:personalize:ap-southeast-1:248025046818:schema/personalize-intertactions-schema1'
customer_schema_arn = 'arn:aws:personalize:ap-southeast-1:248025046818:schema/personalize-customer-schema1'
dataset_group_arn = 'arn:aws:personalize:ap-southeast-1:248025046818:dataset-group/personalize-demo-dataset1'

interactions_dataset_arn = 'arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-demo-dataset1/INTERACTIONS'
customer_dataset_arn = 'arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-demo-dataset1/USERS'

solution_arn ='arn:aws:personalize:ap-southeast-1:248025046818:solution/demo-hrnn-solution1'
campaign_arn = 'arn:aws:personalize:ap-southeast-1:248025046818:campaign/demo-hrnn-campaign1'

# Create Interactions Data 

In [5]:
import json
import csv
from datetime import datetime
from datetime import timedelta
# from datetime import time
from calendar import monthrange
from random import seed
from random import randint
import numpy as np
import pandas as pd
#import platform
# print(platform.python_version())


###################
## Declare constants 
###################
# customer ID start
starting_cust_id=30908

# customer ID end
ending_cust_id=30908+11000

# how many evemts we need?
max_interactions_sequence_count=8000

# starting date of the dataset
event_start_date = datetime(2020,1, 1)
# datetime.datetime.utcfromtimestamp(0)
epoch_event_start_date = int(event_start_date.strftime('%s'))
# print(epoch_event_start_date)

event_time_range = 31536000 #seconds in a year. 

max_time_between_events = 1059200  
min_time_between_events = 3900 # 3 days. 


# lets createa a set of possible interactions 
# Lets say there are 6 different types of intents. 
# each intent can have a sequence of calls of different types
# we will use these fixed set of interaction sequence to generate synthetic data

intent_type=6 # number denoting the intent
interaction_sequence_length = 5 # this number is denoting event type within a sequence


#data column headers 
# USER_ID (string), ITEM_ID (string), TIMESTAMP (long), IMPRESSION


with open('interactions.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(["USER_ID", "ITEM_ID", "TIMESTAMP"])

    ## Loop for interaction sequences 
    interactions_sequence_count=0 
    while ( interactions_sequence_count <= max_interactions_sequence_count ):

        #pic a customer id randomly 
        customer_id = randint(starting_cust_id, ending_cust_id)

        #pic a event sequence randomly 
        intent_id =  randint(0, intent_type-1)

        # select a event sequence length 
        max_sequence_length = randint(0, interaction_sequence_length-1)

        sequence_length = 0
        time_lapse_between_event = epoch_event_start_date+event_time_range
        next_event_time =randint(epoch_event_start_date , time_lapse_between_event)

        while (sequence_length <= max_sequence_length):
            writer.writerow([customer_id, 'intent'+str(intent_id+1)+'_event'+str(sequence_length+1) ,next_event_time])
            sequence_length+=1
            next_event_time += randint(min_time_between_events, max_time_between_events)

        interactions_sequence_count+=1

file.close()


interactions_df=pd.read_csv("interactions.csv")
print(interactions_df.describe())
interactions_df.head(20)

            USER_ID     TIMESTAMP
count  23916.000000  2.391600e+04
mean   36418.960570  1.594131e+09
std     3190.621396  9.103941e+06
min    30909.000000  1.577857e+09
25%    33662.000000  1.586323e+09
50%    36449.500000  1.593990e+09
75%    39189.000000  1.601938e+09
max    41907.000000  1.612628e+09


,USER_ID,ITEM_ID,TIMESTAMP
0,32355,intent2_event1,1609257960
1,39202,intent3_event1,1596533905
2,39202,intent3_event2,1597135186
3,39202,intent3_event3,1597615886
4,34222,intent3_event1,1583998844
5,34222,intent3_event2,1584989732
6,34222,intent3_event3,1585505538
7,32471,intent2_event1,1586787614
8,32471,intent2_event2,1586874725
9,32471,intent2_event3,1587051730


## Upload the interactions data to S3

In [6]:
bucket ='personalize-custdata'      # replace with the name of your S3 bucket
filename ='interactions.csv'
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

## Create schema definition for Customer data in Personalize

In [7]:


customer_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "deposit",
            "type": "string"
        },
        {
            "name": "age",
            "type": "long"
        },
        {
            "name": "job",
            "type": "string"
        },
        {
            "name": "marital",
            "type": "string"
        },
        {
            "name": "education",
            "type": "string"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-customer-schema1",
    schema = json.dumps(customer_schema)
)

customer_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))


{
  "schemaArn": "arn:aws:personalize:ap-southeast-1:248025046818:schema/personalize-customer-schema1",
  "ResponseMetadata": {
    "RequestId": "79f40d9c-cd98-4a62-8053-dfe4f58f65ed",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 09:23:00 GMT",
      "x-amzn-requestid": "79f40d9c-cd98-4a62-8053-dfe4f58f65ed",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Create schema definition for Interactions data in Personalize

In [8]:
interactions_schema =  {
  "type": "record",
  "name": "Interactions",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "ITEM_ID",
          "type": "string"
      },
      {
          "name": "TIMESTAMP",
          "type": "long"
      }
  ],
  "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-intertactions-schema1",
    schema = json.dumps(interactions_schema)
)

interactions_schema_arn = create_schema_response['schemaArn']

print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-southeast-1:248025046818:schema/personalize-intertactions-schema1",
  "ResponseMetadata": {
    "RequestId": "60c233b6-cd87-4786-8f49-acdc542b3399",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 09:23:08 GMT",
      "x-amzn-requestid": "60c233b6-cd87-4786-8f49-acdc542b3399",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Createa a dataset group

In [9]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-demo-dataset1"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:ap-southeast-1:248025046818:dataset-group/personalize-demo-dataset1",
  "ResponseMetadata": {
    "RequestId": "ddf50b56-4690-4161-9d1c-528f7bef0af2",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 09:23:12 GMT",
      "x-amzn-requestid": "ddf50b56-4690-4161-9d1c-528f7bef0af2",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [10]:

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: ACTIVE


## Create datasets 

In [12]:
print (dataset_group_arn)
print (interactions_schema_arn)
print (customer_schema_arn)


arn:aws:personalize:ap-southeast-1:248025046818:dataset-group/personalize-demo-dataset1
arn:aws:personalize:ap-southeast-1:248025046818:schema/personalize-intertactions-schema1
arn:aws:personalize:ap-southeast-1:248025046818:schema/personalize-customer-schema1


In [13]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn,
    name = "demo-dataset"
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))


{
  "datasetArn": "arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-demo-dataset1/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "ad01aef8-81a5-4947-bcc0-44dbbf807f12",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 09:26:32 GMT",
      "x-amzn-requestid": "ad01aef8-81a5-4947-bcc0-44dbbf807f12",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [14]:
dataset_type = "USERS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = customer_schema_arn,
    name = "demo-user-dataset"
)

customer_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))


{
  "datasetArn": "arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-demo-dataset1/USERS",
  "ResponseMetadata": {
    "RequestId": "4b3f68ff-719d-4d32-b057-acf6d69dfe5d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 09:26:45 GMT",
      "x-amzn-requestid": "4b3f68ff-719d-4d32-b057-acf6d69dfe5d",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Add S3 bucket policy for accessing from personalize

In [3]:
s3 = boto3.client("s3")
bucket ='personalize-custdata' 

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

## Create Data Import Jobs

In [15]:
bucket ='personalize-custdata' 
role_arn = 'arn:aws:iam::248025046818:role/AmazonPersonalizeRole'
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "interactions-dataset-import-job2",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, 'interactions.csv')
        
    }, roleArn = role_arn
)

interactions_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-southeast-1:248025046818:dataset-import-job/interactions-dataset-import-job2",
  "ResponseMetadata": {
    "RequestId": "8a294255-29a8-4c2f-9df4-77ea59f940d3",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 09:27:15 GMT",
      "x-amzn-requestid": "8a294255-29a8-4c2f-9df4-77ea59f940d3",
      "content-length": "125",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [16]:
role_arn = 'arn:aws:iam::248025046818:role/AmazonPersonalizeRole'
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "user-dataset-import-job2",
    datasetArn = customer_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, 'customer11k.csv')
        
    }, roleArn = role_arn
)

users_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-southeast-1:248025046818:dataset-import-job/user-dataset-import-job2",
  "ResponseMetadata": {
    "RequestId": "8f33f315-2a13-4cbf-80cc-84f3ca83aa4a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 09:27:17 GMT",
      "x-amzn-requestid": "8f33f315-2a13-4cbf-80cc-84f3ca83aa4a",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [17]:
personalize.list_dataset_import_jobs()
#interactions_dataset_import_job_arn = 'arn:aws:personalize:ap-southeast-1:248025046818:dataset-import-job/interactions-dataset-import-job'
#users_dataset_import_job_arn ='arn:aws:personalize:ap-southeast-1:248025046818:dataset-import-job/user-dataset-import-job'

{'datasetImportJobs': [{'datasetImportJobArn': 'arn:aws:personalize:ap-southeast-1:248025046818:dataset-import-job/interactions-dataset-import-job1',
   'jobName': 'interactions-dataset-import-job1',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2020, 9, 9, 11, 39, 53, 156000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 9, 9, 11, 53, 8, 251000, tzinfo=tzlocal())},
  {'datasetImportJobArn': 'arn:aws:personalize:ap-southeast-1:248025046818:dataset-import-job/interactions-dataset-import-job2',
   'jobName': 'interactions-dataset-import-job2',
   'status': 'CREATE IN_PROGRESS',
   'creationDateTime': datetime.datetime(2020, 9, 17, 9, 27, 15, 262000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 9, 17, 9, 27, 16, 735000, tzinfo=tzlocal())},
  {'datasetImportJobArn': 'arn:aws:personalize:ap-southeast-1:248025046818:dataset-import-job/user-dataset-import-job1',
   'jobName': 'user-dataset-import-job1',
   'status': 'ACTIVE',
  

In [18]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = users_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [25]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interactions_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
#     print(dataset_import_job)
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: ACTIVE


In [20]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims
arn:aws:personalize:::recipe/aws-user-personalization


In [21]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

## Create a Solution Defition in Personalize Service

In [22]:
create_solution_response = personalize.create_solution(
    name = 'demo-hrnn-solution1',
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-southeast-1:248025046818:solution/demo-hrnn-solution1",
  "ResponseMetadata": {
    "RequestId": "9979fd16-2dd0-4c38-8ce7-e93ade3b61f7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 09:40:41 GMT",
      "x-amzn-requestid": "9979fd16-2dd0-4c38-8ce7-e93ade3b61f7",
      "content-length": "94",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [23]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-1:248025046818:solution/demo-hrnn-solution1/669d4f2a",
  "ResponseMetadata": {
    "RequestId": "d67ba694-3cbe-4dbf-b25d-48f9e3da6a1c",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 09:40:44 GMT",
      "x-amzn-requestid": "d67ba694-3cbe-4dbf-b25d-48f9e3da6a1c",
      "content-length": "110",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [24]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

In [26]:
create_campaign_response = personalize.create_campaign(
    name = "demo-hrnn-campaign1",
    solutionVersionArn = solution_version_arn,
#     solutionVersionArn = "arn:aws:personalize:ap-southeast-1:248025046818:solution/demo-hrnn-solution1/9f1b2548",
    minProvisionedTPS = 2,  
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-southeast-1:248025046818:campaign/demo-hrnn-campaign1",
  "ResponseMetadata": {
    "RequestId": "d49df992-3949-4975-96ff-fd6d77a1f9d7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 10:24:37 GMT",
      "x-amzn-requestid": "d49df992-3949-4975-96ff-fd6d77a1f9d7",
      "content-length": "94",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [27]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


## Get Recommendations or Intent Prediction using API call

In [10]:
response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = '35151' )

# 37605
# 35151
# 31167

print("Recommended items - score ")
for item in response['itemList']:
     print (item['itemId'], item['score'])

Recommended items - score 
intent4_event4 0.8505303
intent3_event1 0.0392026
intent6_event1 0.0278637
intent2_event1 0.0218449
intent5_event1 0.0171221
intent1_event1 0.016548
intent4_event1 0.0082746
intent4_event2 0.0029215
intent4_event3 0.0024447
intent4_event5 0.0022254
intent1_event2 0.0020647
intent6_event3 0.0011399
intent5_event4 0.0010736
intent2_event2 0.001025
intent3_event4 0.0008981
intent5_event5 0.0008598
intent2_event4 0.0005803
intent3_event2 0.0005519
intent5_event2 0.0003888
intent2_event3 0.0003468
intent3_event3 0.0003196
intent6_event2 0.0002655
intent1_event3 0.0002586
intent1_event5 0.0002413
intent1_event4 0.0002407


## TODO: Add an interaction and then retry the recommendation

In [11]:
event_tracker = 'CallInentTracker'
response = personalize.create_event_tracker(
    name=event_tracker,
    datasetGroupArn=dataset_group_arn
)

tracking_id = response['trackingId']
print(response)

{'eventTrackerArn': 'arn:aws:personalize:ap-southeast-1:248025046818:event-tracker/805bcf28', 'trackingId': '7ecafd66-72ca-4ccf-a3ae-f370948a11c2', 'ResponseMetadata': {'RequestId': '8cc4a2b0-a270-49e5-a99c-19cac83ac405', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Thu, 17 Sep 2020 12:21:56 GMT', 'x-amzn-requestid': '8cc4a2b0-a270-49e5-a99c-19cac83ac405', 'content-length': '144', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [12]:
import random
import string

personalize_events = boto3.client(service_name='personalize-events')

letters = string.ascii_lowercase
session_id= ''.join(random.choice(letters) for i in range(10)) 



In [15]:
personalize_events.put_events(
    trackingId = tracking_id,
    userId= '33874',
    sessionId = session_id,
    eventList = [{
        'sentAt': int(time.time()-604800),
        'eventType': 'call',
        'properties': "{\"itemId\": \"intent1_event1\"}"
        },{
        'sentAt': int(time.time()-404800),
        'eventType': 'call',
        'properties': "{\"itemId\": \"intent1_event2\"}"
        },{
        'sentAt': int(time.time()-94800),
        'eventType': 'call',
        'properties': "{\"itemId\": \"intent1_event3\"}"
        }]
)

{'ResponseMetadata': {'RequestId': '983c07d4-3c39-403e-a0f2-396088a3c9dd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Thu, 17 Sep 2020 12:36:53 GMT',
   'x-amzn-requestid': '983c07d4-3c39-403e-a0f2-396088a3c9dd',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [16]:
response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = '33874' )

print("Recommended items - scores")

for item in response['itemList']:
     print (item['itemId'], item['score'])

Recommended items - scores
intent1_event4 0.9580994
intent6_event1 0.0071509
intent2_event1 0.0056243
intent3_event1 0.0055842
intent1_event1 0.005043
intent1_event3 0.0033258
intent4_event1 0.0030028
intent5_event2 0.0029303
intent5_event1 0.0022991
intent6_event2 0.0009659
intent6_event3 0.000878
intent1_event5 0.0006289
intent1_event2 0.0005966
intent6_event4 0.0004882
intent3_event5 0.000388
intent3_event2 0.0003403
intent6_event5 0.000337
intent4_event5 0.0003104
intent5_event4 0.0002758
intent2_event5 0.000254
intent4_event4 0.0001909
intent4_event3 0.0001902
intent5_event5 0.0001855
intent2_event4 0.0001839
intent3_event4 0.0001744


In [17]:
response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = '41351' )

print("Recommended items - scores")

for item in response['itemList']:
     print (item['itemId'], item['score'])

Recommended items - scores
intent6_event5 0.6788294
intent1_event1 0.064607
intent5_event1 0.0521186
intent6_event1 0.0495295
intent2_event1 0.0491408
intent3_event1 0.0440361
intent4_event1 0.0360562
intent6_event4 0.0081118
intent6_event3 0.0016015
intent5_event2 0.0015939
intent1_event4 0.0012161
intent2_event3 0.0012073
intent2_event5 0.0010761
intent4_event5 0.0009574
intent2_event2 0.0009193
intent3_event5 0.0008952
intent3_event3 0.0008819
intent1_event2 0.0007943
intent2_event4 0.0006941
intent4_event3 0.0006929
intent3_event2 0.0006899
intent5_event3 0.0005767
intent1_event5 0.0005636
intent3_event4 0.0005213
intent5_event5 0.0005141


# TODO: Clean-up

## TODO: Clean-up Personalize Resources

In [4]:
# Delete the campaign 
personalize.delete_campaign(campaignArn=campaign_arn)
time.sleep(60)

In [9]:
personalize.list_campaigns()


{'campaigns': [{'name': 'demo-hrnn-campaign',
   'campaignArn': 'arn:aws:personalize:ap-southeast-1:248025046818:campaign/demo-hrnn-campaign',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2020, 9, 9, 12, 36, 4, 170000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 9, 9, 12, 44, 1, 20000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': 'a4172e43-ee51-44f5-86bd-df7ecf363eef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 17 Sep 2020 09:08:35 GMT',
   'x-amzn-requestid': 'a4172e43-ee51-44f5-86bd-df7ecf363eef',
   'content-length': '228',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [10]:
# Delete the solution
personalize.delete_solution(solutionArn=solution_arn)
time.sleep(60)

In [11]:
personalize.list_solutions()

{'solutions': [{'name': 'demo-hrnn-solution',
   'solutionArn': 'arn:aws:personalize:ap-southeast-1:248025046818:solution/demo-hrnn-solution',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2020, 9, 9, 11, 55, 15, 742000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 9, 9, 11, 55, 15, 742000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': '9f603a18-fa0e-425e-9734-3762928d9555',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 17 Sep 2020 09:10:10 GMT',
   'x-amzn-requestid': '9f603a18-fa0e-425e-9734-3762928d9555',
   'content-length': '230',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [12]:
personalize.list_datasets()

{'datasets': [{'name': 'demo-dataset',
   'datasetArn': 'arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-demo-dataset/INTERACTIONS',
   'datasetType': 'INTERACTIONS',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2020, 9, 9, 11, 38, 38, 148000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 9, 9, 11, 38, 38, 148000, tzinfo=tzlocal())},
  {'name': 'demo-user-dataset',
   'datasetArn': 'arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-demo-dataset/USERS',
   'datasetType': 'USERS',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2020, 9, 9, 11, 38, 42, 368000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 9, 9, 11, 38, 42, 368000, tzinfo=tzlocal())},
  {'name': 'demo-dataset',
   'datasetArn': 'arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-demo-dataset1/INTERACTIONS',
   'datasetType': 'INTERACTIONS',
   'status': 'ACTIVE',
   'creationDateTime': dat

In [13]:
print (customer_dataset_arn)
print (interactions_dataset_arn)

arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-demo-dataset1/USERS
arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-demo-dataset1/INTERACTIONS


In [14]:
# Delete the datasets
personalize.delete_dataset(datasetArn=customer_dataset_arn)
personalize.delete_dataset(datasetArn=interactions_dataset_arn)
time.sleep(60)

In [20]:
personalize.list_datasets()

{'datasets': [{'name': 'demo-dataset',
   'datasetArn': 'arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-demo-dataset/INTERACTIONS',
   'datasetType': 'INTERACTIONS',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2020, 9, 9, 11, 38, 38, 148000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 9, 9, 11, 38, 38, 148000, tzinfo=tzlocal())},
  {'name': 'demo-user-dataset',
   'datasetArn': 'arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-demo-dataset/USERS',
   'datasetType': 'USERS',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2020, 9, 9, 11, 38, 42, 368000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 9, 9, 11, 38, 42, 368000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': '88146834-1403-4c91-91fd-cf3d68637a9b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 17 Sep 2020 09:13:26 GMT',
   'x-amzn-reque

In [21]:
print(dataset_group_arn)

arn:aws:personalize:ap-southeast-1:248025046818:dataset-group/personalize-demo-dataset1


In [22]:
#Delete dataset group
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

{'ResponseMetadata': {'RequestId': '067d324c-cc36-4b1b-90a6-83a6d69ea7de',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 17 Sep 2020 09:13:45 GMT',
   'x-amzn-requestid': '067d324c-cc36-4b1b-90a6-83a6d69ea7de',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [24]:
personalize.list_dataset_groups()

{'datasetGroups': [{'name': 'personalize-demo-dataset',
   'datasetGroupArn': 'arn:aws:personalize:ap-southeast-1:248025046818:dataset-group/personalize-demo-dataset',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2020, 9, 9, 11, 34, 21, 193000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 9, 9, 11, 34, 33, 665000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': '3988a893-3380-4f56-aa00-ea1167cef8d9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 17 Sep 2020 09:14:56 GMT',
   'x-amzn-requestid': '3988a893-3380-4f56-aa00-ea1167cef8d9',
   'content-length': '255',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [25]:
# Delete the schema
personalize.delete_schema(schemaArn=customer_schema_arn)
personalize.delete_schema(schemaArn=interactions_schema_arn)
time.sleep(60)

In [26]:
personalize.list_schemas()

{'schemas': [{'name': 'personalize-customer-schema',
   'schemaArn': 'arn:aws:personalize:ap-southeast-1:248025046818:schema/personalize-customer-schema',
   'creationDateTime': datetime.datetime(2020, 9, 9, 11, 32, 17, 29000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 9, 9, 11, 32, 17, 29000, tzinfo=tzlocal())},
  {'name': 'personalize-intertactions-schema',
   'schemaArn': 'arn:aws:personalize:ap-southeast-1:248025046818:schema/personalize-intertactions-schema',
   'creationDateTime': datetime.datetime(2020, 9, 9, 11, 34, 14, 900000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 9, 9, 11, 34, 14, 900000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': 'ddcac935-9c10-497d-9252-a793bf48b610',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 17 Sep 2020 09:16:03 GMT',
   'x-amzn-requestid': 'ddcac935-9c10-497d-9252-a793bf48b610',
   'content-length': '441',
   'connection': 

## TODO: Clean-up Data